In [ ]:
# Neural Style Transfer Web Application
# File: neural_style_transfer_webapp.py

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from flask import Flask, request, jsonify, render_template_string
import base64
import io
from PIL import Image
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class NeuralStyleTransfer:
    def __init__(self, content_layers=['block5_conv2'],
                 style_layers=['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']):
        """
        Initialize Neural Style Transfer model

        Args:
            content_layers: Layers to extract content features
            style_layers: Layers to extract style features
        """
        self.content_layers = content_layers
        self.style_layers = style_layers
        self.num_content_layers = len(content_layers)
        self.num_style_layers = len(style_layers)

        # Build the VGG model
        self.vgg = self.vgg_layers(self.style_layers + self.content_layers)
        self.vgg.trainable = False

        # Style and content weights
        self.style_weight = 1e-2
        self.content_weight = 1e4
        self.total_variation_weight = 30

    def vgg_layers(self, layer_names):
        """Create a VGG model that returns feature maps for specified layers"""
        vgg = VGG19(include_top=False, weights='imagenet')
        vgg.trainable = False

        outputs = [vgg.get_layer(name).output for name in layer_names]
        model = tf.keras.Model([vgg.input], outputs)
        return model

    def preprocess_image(self, image_path, max_dim=512):
        """Load and preprocess image"""
        img = load_img(image_path)
        img = tf.image.convert_image_dtype(img, tf.float32)

        shape = tf.cast(tf.shape(img)[:-1], tf.float32)
        long_dim = max(shape)
        scale = max_dim / long_dim

        new_shape = tf.cast(shape * scale, tf.int32)
        img = tf.image.resize(img, new_shape)
        img = img[tf.newaxis, :]
        return img

    def deprocess_image(self, processed_img):
        """Convert processed image back to displayable format"""
        x = processed_img.copy()
        if len(x.shape) == 4:
            x = np.squeeze(x, 0)

        # Remove zero-center by mean pixel
        x[:, :, 0] += 103.939
        x[:, :, 1] += 116.779
        x[:, :, 2] += 123.68

        # Convert BGR to RGB
        x = x[:, :, ::-1]
        x = np.clip(x, 0, 255).astype('uint8')
        return x

    def gram_matrix(self, input_tensor):
        """Calculate Gram matrix for style representation"""
        result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
        input_shape = tf.shape(input_tensor)
        num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
        return result/(num_locations)

    def style_content_loss(self, outputs, style_targets, content_targets):
        """Calculate style and content loss"""
        style_outputs = outputs['style']
        content_outputs = outputs['content']

        style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2)
                               for name in style_outputs.keys()])
        style_loss *= self.style_weight / self.num_style_layers

        content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2)
                                 for name in content_outputs.keys()])
        content_loss *= self.content_weight / self.num_content_layers

        loss = style_loss + content_loss
        return loss

    def extract_features(self, inputs):
        """Extract style and content features"""
        inputs = inputs * 255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)

        style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                          outputs[self.num_style_layers:])

        style_outputs = [self.gram_matrix(style_output) for style_output in style_outputs]

        content_dict = {content_name: value
                        for content_name, value in zip(self.content_layers, content_outputs)}

        style_dict = {style_name: value
                      for style_name, value in zip(self.style_layers, style_outputs)}

        return {'content': content_dict, 'style': style_dict}

    def high_pass_x_y(self, image):
        """High pass filter for total variation loss"""
        x_var = image[:, :, 1:, :] - image[:, :, :-1, :]
        y_var = image[:, 1:, :, :] - image[:, :-1, :, :]
        return x_var, y_var

    def total_variation_loss(self, image):
        """Calculate total variation loss for smoothness"""
        x_deltas, y_deltas = self.high_pass_x_y(image)
        return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))

    @tf.function()
    def train_step(self, image, style_targets, content_targets, optimizer):
        """Single training step"""
        with tf.GradientTape() as tape:
            outputs = self.extract_features(image)
            loss = self.style_content_loss(outputs, style_targets, content_targets)
            loss += self.total_variation_weight * tf.nn.l2_loss(self.total_variation_loss(image))

        grad = tape.gradient(loss, image)
        optimizer.apply_gradients([(grad, image)])
        image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0))
        return loss

    def transfer_style(self, content_path, style_path, epochs=10, steps_per_epoch=100):
        """Main style transfer function"""
        # Load and preprocess images
        content_image = self.preprocess_image(content_path)
        style_image = self.preprocess_image(style_path)

        # Extract style and content targets
        style_targets = self.extract_features(style_image)['style']
        content_targets = self.extract_features(content_image)['content']

        # Initialize generated image
        image = tf.Variable(content_image)

        # Optimizer
        opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

        # Training loop
        for epoch in range(epochs):
            for step in range(steps_per_epoch):
                loss = self.train_step(image, style_targets, content_targets, opt)

            if epoch % 2 == 0:
                logger.info(f"Epoch {epoch}, Loss: {loss:.4f}")

        return image.numpy()

# Flask Web Application
app = Flask(__name__)

# HTML Template
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Neural Style Transfer</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; background: #f5f5f5; }
        .container { max-width: 800px; margin: 0 auto; background: white; padding: 30px; border-radius: 10px; }
        h1 { color: #333; text-align: center; }
        .upload-section { margin: 20px 0; padding: 20px; border: 2px dashed #ddd; border-radius: 5px; }
        .result-section { margin: 20px 0; text-align: center; }
        input[type=file] { margin: 10px 0; }
        button { background: #007bff; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer; }
        button:hover { background: #0056b3; }
        .loading { display: none; text-align: center; }
        img { max-width: 100%; height: auto; margin: 10px; border-radius: 5px; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎨 Neural Style Transfer</h1>

        <div class="upload-section">
            <h3>Upload Content Image</h3>
            <input type="file" id="contentImage" accept="image/*">
            <div id="contentPreview"></div>
        </div>

        <div class="upload-section">
            <h3>Upload Style Image</h3>
            <input type="file" id="styleImage" accept="image/*">
            <div id="stylePreview"></div>
        </div>

        <button onclick="transferStyle()">Generate Stylized Image</button>

        <div class="loading" id="loading">
            <h3>Processing... This may take a few minutes.</h3>
            <div>🎨 Applying artistic style...</div>
        </div>

        <div class="result-section" id="result"></div>
    </div>

    <script>
        function previewImage(input, previewId) {
            const file = input.files[0];
            if (file) {
                const reader = new FileReader();
                reader.onload = function(e) {
                    document.getElementById(previewId).innerHTML =
                        '<img src="' + e.target.result + '" style="max-width: 300px;">';
                };
                reader.readAsDataURL(file);
            }
        }

        document.getElementById('contentImage').addEventListener('change', function() {
            previewImage(this, 'contentPreview');
        });

        document.getElementById('styleImage').addEventListener('change', function() {
            previewImage(this, 'stylePreview');
        });

        async function transferStyle() {
            const contentFile = document.getElementById('contentImage').files[0];
            const styleFile = document.getElementById('styleImage').files[0];

            if (!contentFile || !styleFile) {
                alert('Please upload both images!');
                return;
            }

            const formData = new FormData();
            formData.append('content', contentFile);
            formData.append('style', styleFile);

            document.getElementById('loading').style.display = 'block';
            document.getElementById('result').innerHTML = '';

            try {
                const response = await fetch('/transfer', {
                    method: 'POST',
                    body: formData
                });

                const data = await response.json();

                if (data.success) {
                    document.getElementById('result').innerHTML =
                        '<h3>Result:</h3><img src="data:image/png;base64,' + data.result + '">';
                } else {
                    alert('Error: ' + data.error);
                }
            } catch (error) {
                alert('Error processing images: ' + error.message);
            }

            document.getElementById('loading').style.display = 'none';
        }
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/transfer', methods=['POST'])
def transfer_style():
    try:
        # Get uploaded files
        content_file = request.files['content']
        style_file = request.files['style']

        # Save temporary files
        content_path = 'temp_content.jpg'
        style_path = 'temp_style.jpg'

        content_file.save(content_path)
        style_file.save(style_path)

        # Initialize style transfer model
        nst = NeuralStyleTransfer()

        # Perform style transfer
        result_image = nst.transfer_style(content_path, style_path, epochs=5, steps_per_epoch=50)

        # Convert result to base64
        result_image = nst.deprocess_image(result_image)
        pil_image = Image.fromarray(result_image)

        buffer = io.BytesIO()
        pil_image.save(buffer, format='PNG')
        img_str = base64.b64encode(buffer.getvalue()).decode()

        # Clean up temporary files
        os.remove(content_path)
        os.remove(style_path)

        return jsonify({'success': True, 'result': img_str})

    except Exception as e:
        logger.error(f"Error in style transfer: {str(e)}")
        return jsonify({'success': False, 'error': str(e)})

@app.route('/health')
def health_check():
    return jsonify({'status': 'healthy', 'model': 'neural_style_transfer'})

if __name__ == '__main__':
    # Ensure TensorFlow uses CPU for compatibility
    tf.config.set_visible_devices([], 'GPU')

    print("🎨 Neural Style Transfer Web App Starting...")
    print("📱 Access the app at: http://localhost:5000")
    print("🔧 API endpoint: http://localhost:5000/transfer")

    app.run(debug=True, host='0.0.0.0', port=5000)

🎨 Neural Style Transfer Web App Starting...
📱 Access the app at: http://localhost:5000
🔧 API endpoint: http://localhost:5000/transfer
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
